In [1]:
import requests
from bs4 import BeautifulSoup as bs
import lxml
import json
import constant
import ast
import re

In [3]:
def get_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'}
    r = requests.get(url, headers = headers)
    return r

In [4]:
 r = get_html('https://www.fiesta.ru/spb/novelty/events/')

In [5]:
print(r)

<Response [200]>


In [ ]:
id_parse,type_event, , , date_start, date_stop, cost, discounted, address, metro, 

In [45]:
html = bs(r.text, 'html.parser')
event = []
soup = html.find_all('div', class_="grid_i grid_i__desktop-grid-1-3 grid_i__tablet-grid-1-2 grid_i__phone-grid-1-1")
#s = soup[2]
for s in soup:
    discounted = 0
    id_parse = s.find('footer', class_='unit_f visible-desktop').attrs['data-calendar-item']
    title = s.find('a', class_='unit_t_a double-hover').get_text()
    link = 'https://www.fiesta.ru/' + s.find('a', class_='unit_t_a double-hover').get('href')
    img = 'https://www.fiesta.ru/' + s.find('img').get('src')
    # получаем дату события
    date = s.find('p', class_='unit_date').get_text()
    if date.find('–') > 0:
        try:
            date_array = date.split('–')
            date_start = date_array[0].strip()
            date_stop = date_array[1].strip()
        except:
            date_start = None
            date_stop = None
    elif date.find(','):
        try:
            date_array = date.split(',')
            date_start = date_array[0].strip()
            date_stop = date_array[-1].strip()
        except:
            date_start = None
            date_stop = None
    if date_start == date_stop:
        date_stop = None
    try:
        place = s.find('p', class_='unit_place').get_text()
        if place.startswith('м.'):
            place_array = place.split(',')
            metro = place_array[0].strip()
            address = ','.join(place_array[1:]).strip()
        else:
            metro = None
            address = None
    except:
        metro = None
        address = None
    link_cost = get_html(link)
    html_event = bs(r.text, 'html.parser')
    #cost = html_event.find('dd', class_='grid_i grid_i__desktop-grid-5-6 grid_i__tablet-grid-5-6 grid_i__phone-grid-1-1').get_text()

    print(id_parse, title, link, img, date_start, date_stop, '\n', metro, address, sep=' ')
    print('\n')
    

46825 Международная фотовыставка «Материнство в дикой природе» https://www.fiesta.ru//spb/events/mezhdunarodnaya-fotovystavka-materinstvo-v-dikoy-prirode/ https://www.fiesta.ru//uploads/slider_image/image/182037/slider_carousel_thumb_jBIbsLWU-SE.jpg 4 апреля 15 апреля 
 None None


46813 Выставка «Убежище. Welcome to the paradise» https://www.fiesta.ru//spb/events/vystavka-ubezhische-welcome-to-the-paradise/ https://www.fiesta.ru//uploads/slider_image/image/182008/slider_carousel_thumb_DSC_8546.jpg 4 апреля 23 мая 
 м. Ладожская шоссе Революции, 84АБ


30088 Вечеринки романтических знакомств от Dating-City  https://www.fiesta.ru//spb/events/vecherinki-romanticheskih-znakomstv-ot-dating-city/ https://www.fiesta.ru//uploads/slider_image/image/156426/slider_carousel_thumb_%D0%B1%D1%8119.jpg 9 апреля 11 апреля 
 None None


46394 «Гаражная распродажа в Порту» https://www.fiesta.ru//spb/events/garazhnaya-rasprodazha-v-portu/ https://www.fiesta.ru//uploads/slider_image/image/180465/slider_ca

In [46]:
link_cost = get_html('https://www.fiesta.ru//spb/events/shou-sem-hronika-lyubovnyh-proisshestviy/')
html_event = bs(r.text, 'html.parser')
cost = html_event.find('dd', class_='grid_i grid_i__desktop-grid-5-6 grid_i__tablet-grid-5-6 grid_i__phone-grid-1-1')
print(cost)

None


In [171]:
event = []
id_parse = html_event['id']
# получем минимальную и максимальную стоимость события
min_cost = html_event['minPrice']
max_cost = html_event['maxPrice']
# формируем стоимость
if min_cost == max_cost:
    cost = str(max_cost) + ' рублей'
else:
    cost = 'от ' + str(min_cost) + ' до ' + str(max_cost) + ' рублей'
# получаем дату события
date = html_event['date']
# если событие не один день
if type(date) == dict:
    date_start = re_format_kudago(html_event['date']['start_min'])
    date_stop = re_format_kudago(html_event['date']['start_max'])
else:
    date_start = re_format_kudago(date)
    date_stop = None
type_event = "Концерт"
# вшитый в страницу сктипт с параметрами события
dictonary_event = json.loads(str(s.find('script', type="application/ld+json").string))
img = dictonary_event['image']
title = dictonary_event['name']
discounted = 0
address_place = dictonary_event['location']['address']
# убираем Санк-Петербург из адреса
if address_place.startswith('Санкт'):
    address_spb = address_place.split(', ')
    address_place = ', '.join(address_spb[1:])
address = dictonary_event['location']['name'] + ", " + address_place
metro = None
link = dictonary_event['url']
event.insert(0, (id_parse,type_event, img, title, date_start, date_stop, cost, discounted, address, metro, link))
print(event)

[(12257, 'Концерт', 'https://cdn.kassir.ru/kassir.ru/poster/7f/7f8e0525bf368e6f0bd8b1073be7ca4e.jpg', 'Кентервильское привидение', '28 марта 2021', '25 апреля 2021', 'от 600 до 800 рублей', 0, 'Театр Мастерская, ул. Народная, дом 1', None, 'https://spb.kassir.ru/koncert/teatr-masterskaya/kentervilskoe-prividenie_2021-03-28')]


In [155]:
def re_format_kudago(date):
    if len(date) > 10:
        # '2021-04-28 20:00:00'
        date =  date[0:10]
    list_date = date.split('-')
# удаление нуля в числе 
    number = int(list_date[2])
    re_format_date = str(number) + " " + constant.date_format_kudago[int(list_date[1])-1] + " " + list_date[0]
    return re_format_date

In [150]:
x = '2021-04-28 20:00:00'
y =  x[0:10]
print(y)

2021-04-28


In [127]:
x = '{"name":"Гречанинов. Демественная литургия","id":778220,"price":300,"category":"Театр","category_id":3009,"image":"https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg","venueName":"Государственная академическая Капелла","position":1,"cat1":"Театр","cat2":"","eventId":778220,"eventName":"Гречанинов. Демественная литургия","minPrice":300,"maxPrice":450,"date":"2021-03-31 19:00:00","avail":390,"placementId":107}'


print(x)

{"name":"Гречанинов. Демественная литургия","id":778220,"price":300,"category":"Театр","category_id":3009,"image":"https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg","venueName":"Государственная академическая Капелла","position":1,"cat1":"Театр","cat2":"","eventId":778220,"eventName":"Гречанинов. Демественная литургия","minPrice":300,"maxPrice":450,"date":"2021-03-31 19:00:00","avail":390,"placementId":107}


In [128]:
w = re.sub(r'[^:]""',' ', x).replace(' "', ' ').replace('" ', ' ')
print(w)
y = x.replace('""', '"').replace(' "', ' ').replace('" ', ' ')


{"name":"Гречанинов. Демественная литургия","id":778220,"price":300,"category":"Театр","category_id":3009,"image":"https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg","venueName":"Государственная академическая Капелла","position":1,"cat1":"Театр","cat2":"","eventId":778220,"eventName":"Гречанинов. Демественная литургия","minPrice":300,"maxPrice":450,"date":"2021-03-31 19:00:00","avail":390,"placementId":107}


In [129]:
json.loads(w)

{'name': 'Гречанинов. Демественная литургия',
 'id': 778220,
 'price': 300,
 'category': 'Театр',
 'category_id': 3009,
 'image': 'https://cdn.kassir.ru/spb/poster/97/97ae0a5b8b109a05ac1ce8fae69f4058.jpg',
 'venueName': 'Государственная академическая Капелла',
 'position': 1,
 'cat1': 'Театр',
 'cat2': '',
 'eventId': 778220,
 'eventName': 'Гречанинов. Демественная литургия',
 'minPrice': 300,
 'maxPrice': 450,
 'date': '2021-03-31 19:00:00',
 'avail': 390,
 'placementId': 107}